In [1]:
# importing necessary modules for data cleaning and preprocessing

import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

In [2]:
# converting data in dataframe

df = pd.read_csv('E:\sales_analysis\datasets\online_retail.csv')

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6008\3562321399.py:3: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv('E:\sales_analysis\datasets\online_retail.csv')


In [3]:
# viewing the random rows of the dataframe
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/09 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/09 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/09 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/09 7:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/09 7:45,1.25,13085.0,United Kingdom



Observation :

-> There are few data points that shows cancellation or return of any certain product whose invoice code starts with C, but in few datapoints, the customerID is missing! So we can not use those data points for customer segmentation however we can still use it for stock suggestion as how much return/cancellation rate does a product have!

-> Description of few products are missing which definitely won't effect the ml model as we don't neccesarily need the description, but we can fix it by checking if there is any other sale with same product code and filling the gap! (by finding the mode of 'Description' of same stock code)

-> Few invoices start with some other letter like 'A' for adjustment which doesn't tell me alot.

-> Few products were sold at negative quantity but their invoices dont start with 'C' which might mean damages or gifts or ebay sales or miscelleneous, which means they are outlliers.


<h4>Invoice</h4>
<ul>
<li> Starts with 'C' : means cacellation or return of a particular product
<li> Starts with 'A' : means bad debt adjustments
</ul>

In [4]:
# Number of unique products

unique_products = df["StockCode"].unique()

print("TOTAL UNIQUE PRODUCTS :", len(unique_products))

TOTAL UNIQUE PRODUCTS : 4632


Objective : filling Description of products with missing description

Method :

-> finding the data point with missing description

-> fetching its stockcode

-> fetching description of all the datapoints with same stock code

-> finding mode of set of all descriptions with required stock code

-> filling the mode value at the missing place

-> if there is no other datapoint with same stock id, then remove it!

-> in the end check if there is any datapoint missing

-> repeat the whole process until there is no NaN in Description

column name for stock code is "StockCode" and description is "Description"

In [5]:
def fill_missing_descriptions(df):
    
    # Create a copy to avoid modifying the original dataframe
    df_copy = df.copy()
    
    print(f"Starting with {len(df_copy)} total records")
    print(f"Missing descriptions: {df_copy['Description'].isna().sum()}")
    
    iteration = 0
    
    # Repeat until no missing descriptions remain
    while df_copy['Description'].isna().any():
        iteration += 1
        print(f"\n--- Iteration {iteration} ---")
        
        # Find datapoints with missing descriptions
        missing_mask = df_copy['Description'].isna()
        missing_stockcodes = df_copy[missing_mask]['StockCode'].unique()
        
        print(f"Stock codes with missing descriptions: {len(missing_stockcodes)}")
        
        # Track which rows to remove (no other datapoints with same stock code)
        rows_to_remove = []
        filled_count = 0
        
        for stockcode in missing_stockcodes:
            # Get all descriptions for this stock code (excluding NaN)
            stockcode_descriptions = df_copy[
                (df_copy['StockCode'] == stockcode) & 
                (df_copy['Description'].notna())
            ]['Description']
            
            if len(stockcode_descriptions) == 0:
                # No other datapoints with this stock code - mark for removal
                rows_to_remove.extend(
                    df_copy[
                        (df_copy['StockCode'] == stockcode) & 
                        (df_copy['Description'].isna())
                    ].index.tolist()
                )
                print(f"  Stock code {stockcode}: No other descriptions found - marking for removal")
            
            else:
                # Find mode (most frequent description) for this stock code
                try:
                    # Count occurrences of each description
                    description_counts = stockcode_descriptions.value_counts()
                    
                    # Get the most frequent description (mode)
                    mode_description = description_counts.index[0]
                    mode_count = description_counts.iloc[0]
                    
                    # Fill missing descriptions with the mode
                    mask = (df_copy['StockCode'] == stockcode) & (df_copy['Description'].isna())
                    df_copy.loc[mask, 'Description'] = mode_description
                    
                    filled_this_stockcode = mask.sum()
                    filled_count += filled_this_stockcode
                    
                    print(f"  Stock code {stockcode}: Filled {filled_this_stockcode} missing descriptions with '{mode_description}' (appeared {mode_count} times)")
                
                except Exception as e:
                    print(f"  Error processing stock code {stockcode}: {e}")
        
        # Remove rows that can't be filled
        if rows_to_remove:
            df_copy = df_copy.drop(rows_to_remove)
            print(f"  Removed {len(rows_to_remove)} rows with no fillable descriptions")
        
        print(f"  Filled {filled_count} descriptions in this iteration")
        print(f"  Remaining missing descriptions: {df_copy['Description'].isna().sum()}")
        print(f"  Total records remaining: {len(df_copy)}")
        
        # Safety check to prevent infinite loops
        if iteration > 100:
            print("\nWarning: Maximum iterations reached. Breaking to prevent infinite loop.")
            break
    
    print(f"\n=== Final Results ===")
    print(f"Total iterations: {iteration}")
    print(f"Final record count: {len(df_copy)}")
    print(f"Remaining missing descriptions: {df_copy['Description'].isna().sum()}")
    print(f"Records removed: {len(df) - len(df_copy)}")
    
    return df_copy

def analyze_descriptions_by_stockcode(df):
    """
    Analyze the distribution of descriptions by stock code to understand the data.
    
    Parameters:
    df (pandas.DataFrame): DataFrame with 'StockCode' and 'Description' columns
    """
    print("=== Description Analysis by Stock Code ===")
    
    # Group by stock code and analyze descriptions
    stockcode_analysis = df.groupby('StockCode')['Description'].agg([
        'count',
        'nunique',
        lambda x: x.isna().sum()
    ]).rename(columns={'<lambda_0>': 'missing_count'})
    
    stockcode_analysis['non_missing_count'] = stockcode_analysis['count'] - stockcode_analysis['missing_count']
    
    print(f"Total unique stock codes: {len(stockcode_analysis)}")
    print(f"Stock codes with all missing descriptions: {(stockcode_analysis['non_missing_count'] == 0).sum()}")
    print(f"Stock codes with some missing descriptions: {(stockcode_analysis['missing_count'] > 0).sum()}")
    print(f"Stock codes with multiple unique descriptions: {(stockcode_analysis['nunique'] > 1).sum()}")
    
    return stockcode_analysis

# Example usage and testing
if __name__ == "__main__":
    
    
    # Load your data
    df = pd.read_csv('E:\sales_analysis\datasets\online_retail.csv')  # or however you load your data
    
    # Optional: Analyze the data first
    analysis = analyze_descriptions_by_stockcode(df)
    
    # Fill missing descriptions
    df_cleaned = fill_missing_descriptions(df)
    
    # Save the result
    df_cleaned.to_csv('E:\sales_analysis\datasets\cleaned_data_with_filled_descriptions.csv', index=False)


<>:117: SyntaxWarning: invalid escape sequence '\s'
<>:126: SyntaxWarning: invalid escape sequence '\s'
<>:117: SyntaxWarning: invalid escape sequence '\s'
<>:126: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6008\3290892807.py:117: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv('E:\sales_analysis\datasets\online_retail.csv')  # or however you load your data
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6008\3290892807.py:126: SyntaxWarning: invalid escape sequence '\s'
  df_cleaned.to_csv('E:\sales_analysis\datasets\cleaned_data_with_filled_descriptions.csv', index=False)


=== Description Analysis by Stock Code ===
Total unique stock codes: 4632
Stock codes with all missing descriptions: 83
Stock codes with some missing descriptions: 1920
Stock codes with multiple unique descriptions: 687
Starting with 525461 total records
Missing descriptions: 2928

--- Iteration 1 ---
Stock codes with missing descriptions: 1920
  Stock code 21646: No other descriptions found - marking for removal
  Stock code 20683: Filled 2 missing descriptions with 'RAIN GIRL CHILDS UMBRELLA' (appeared 1 times)
  Stock code 21350: Filled 3 missing descriptions with 'GINGHAM HEART WREATH' (appeared 46 times)
  Stock code 84292: Filled 3 missing descriptions with 'ROSE PINK METAL FOLDING CHAIR' (appeared 15 times)
  Stock code 18010: Filled 2 missing descriptions with 'ASSORTED FRAGRANCE BATH CONFETTI' (appeared 9 times)
  Stock code 85049G: Filled 2 missing descriptions with 'CHOCOLATE BOX RIBBONS ' (appeared 274 times)
  Stock code 35751C: Filled 1 missing descriptions with 'PURPLE C

Objective 2 : Creating a new dataset 

Columns to keep in new dataset : Invoice, StockCode, Description, Qunatity, InvoiceDate, Price

Method : 

-> Create a new dataset named "sales_data.csv" 

    designing of Schema of sales_data.csv!

    i have an existing dataset that i have attached, now i want a dataset with monthly and weekly quantity sold and, monthly and weekly total price of each unique stockcode with description of each code (remember that the column "Price" in attached dataset is per unit price then the net price will be quantity * price) , write me python code for that
 
-> Add all the given columns from 

In [6]:
def create_aggregated_datasets(df, start_date='01/12/2009', end_date='31/12/2010'):
    """
    Create aggregated datasets with monthly and weekly sales data for each product.
    Fills missing periods with zero values for all products.
    
    Parameters:
    df (pandas.DataFrame): Original retail transactions dataset
    start_date (str): Start date in MM/DD/YY format
    end_date (str): End date in MM/DD/YY format
    
    Returns:
    tuple: (monthly_agg, weekly_agg) - Two DataFrames with aggregated data
    """
    
    # Create a copy and ensure proper data types
    df_clean = df.copy()
    
    # Convert InvoiceDate to datetime (handling MM/DD/YY Time format)
    df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'], format='%m/%d/%y %H:%M')
    
    # Calculate net price (total price per transaction line)
    df_clean['NetPrice'] = df_clean['Quantity'] * df_clean['Price']
    
    # Create period columns for grouping
    df_clean['YearMonth'] = df_clean['InvoiceDate'].dt.to_period('M')
    df_clean['YearWeek'] = df_clean['InvoiceDate'].dt.to_period('W')
    
    print(f"Processing {len(df_clean)} transactions...")
    print(f"Date range: {df_clean['InvoiceDate'].min()} to {df_clean['InvoiceDate'].max()}")
    print(f"Unique products: {df_clean['StockCode'].nunique()}")
    
    # Get unique StockCode-Description mapping
    product_mapping = df_clean.groupby('StockCode')['Description'].agg(
        lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0]
    ).reset_index()
    
    # Create complete date ranges
    start_dt = datetime.strptime(start_date, '%d/%m/%Y')
    end_dt = datetime.strptime(end_date, '%d/%m/%Y')
    
    # Generate all months and weeks in the range
    all_months = pd.period_range(start=start_dt, end=end_dt, freq='M')
    all_weeks = pd.period_range(start=start_dt, end=end_dt, freq='W')
    
    print(f"Complete period range: {start_dt.date()} to {end_dt.date()}")
    print(f"Total months to cover: {len(all_months)}")
    print(f"Total weeks to cover: {len(all_weeks)}")
    
    # === MONTHLY AGGREGATION ===
    print("\nCreating monthly aggregation...")
    
    # Aggregate existing data by month
    monthly_existing = df_clean.groupby(['StockCode', 'YearMonth']).agg({
        'Quantity': 'sum',
        'NetPrice': 'sum'
    }).reset_index()
    
    # Create complete grid: all products × all months
    monthly_grid = pd.MultiIndex.from_product([
        product_mapping['StockCode'].unique(),
        all_months
    ], names=['StockCode', 'YearMonth']).to_frame(index=False)
    
    # Merge with existing data
    monthly_agg = monthly_grid.merge(monthly_existing, on=['StockCode', 'YearMonth'], how='left')
    
    # Fill missing values with 0
    monthly_agg['Quantity'] = monthly_agg['Quantity'].fillna(0)
    monthly_agg['NetPrice'] = monthly_agg['NetPrice'].fillna(0)
    
    # Calculate per unit price (avoid division by zero)
    monthly_agg['Monthly_Per_Unit_Price'] = np.where(
        monthly_agg['Quantity'] > 0,
        monthly_agg['NetPrice'] / monthly_agg['Quantity'],
        0
    )
    
    # Rename columns for clarity
    monthly_agg = monthly_agg.rename(columns={
        'Quantity': 'Monthly_Quantity_Sold',
        'NetPrice': 'Monthly_Total_Price'
    })
    
    # Add Description
    monthly_agg = monthly_agg.merge(product_mapping, on='StockCode', how='left')
    
    # Reorder columns
    monthly_agg = monthly_agg[['StockCode', 'Description', 'YearMonth', 
                              'Monthly_Quantity_Sold', 'Monthly_Total_Price', 'Monthly_Per_Unit_Price']]
    
    print(f"Monthly aggregation created: {len(monthly_agg)} records")
    print(f"Records with zero sales: {(monthly_agg['Monthly_Quantity_Sold'] == 0).sum()}")
    
    # === WEEKLY AGGREGATION ===
    print("\nCreating weekly aggregation...")
    
    # Aggregate existing data by week
    weekly_existing = df_clean.groupby(['StockCode', 'YearWeek']).agg({
        'Quantity': 'sum',
        'NetPrice': 'sum'
    }).reset_index()
    
    # Create complete grid: all products × all weeks
    weekly_grid = pd.MultiIndex.from_product([
        product_mapping['StockCode'].unique(),
        all_weeks
    ], names=['StockCode', 'YearWeek']).to_frame(index=False)
    
    # Merge with existing data
    weekly_agg = weekly_grid.merge(weekly_existing, on=['StockCode', 'YearWeek'], how='left')
    
    # Fill missing values with 0
    weekly_agg['Quantity'] = weekly_agg['Quantity'].fillna(0)
    weekly_agg['NetPrice'] = weekly_agg['NetPrice'].fillna(0)
    
    # Calculate per unit price (avoid division by zero)
    weekly_agg['Weekly_Per_Unit_Price'] = np.where(
        weekly_agg['Quantity'] > 0,
        weekly_agg['NetPrice'] / weekly_agg['Quantity'],
        0
    )
    
    # Rename columns for clarity
    weekly_agg = weekly_agg.rename(columns={
        'Quantity': 'Weekly_Quantity_Sold',
        'NetPrice': 'Weekly_Total_Price'
    })
    
    # Add Description
    weekly_agg = weekly_agg.merge(product_mapping, on='StockCode', how='left')
    
    # Reorder columns
    weekly_agg = weekly_agg[['StockCode', 'Description', 'YearWeek', 
                            'Weekly_Quantity_Sold', 'Weekly_Total_Price', 'Weekly_Per_Unit_Price']]
    
    print(f"Weekly aggregation created: {len(weekly_agg)} records")
    print(f"Records with zero sales: {(weekly_agg['Weekly_Quantity_Sold'] == 0).sum()}")
    
    return monthly_agg, weekly_agg

def analyze_aggregated_data(monthly_agg, weekly_agg):
    """
    Provide insights about the aggregated datasets.
    """
    
    print("\n=== AGGREGATION ANALYSIS ===")
    
    print(f"\nMonthly Data:")
    print(f"- Total product-month combinations: {len(monthly_agg)}")
    print(f"- Unique products: {monthly_agg['StockCode'].nunique()}")
    print(f"- Date range: {monthly_agg['YearMonth'].min()} to {monthly_agg['YearMonth'].max()}")
    print(f"- Records with sales: {(monthly_agg['Monthly_Quantity_Sold'] > 0).sum()}")
    print(f"- Records with zero sales: {(monthly_agg['Monthly_Quantity_Sold'] == 0).sum()}")
    print(f"- Total quantity sold: {monthly_agg['Monthly_Quantity_Sold'].sum():,.0f}")
    print(f"- Total revenue: £{monthly_agg['Monthly_Total_Price'].sum():,.2f}")
    
    print(f"\nWeekly Data:")
    print(f"- Total product-week combinations: {len(weekly_agg)}")
    print(f"- Unique products: {weekly_agg['StockCode'].nunique()}")
    print(f"- Date range: {weekly_agg['YearWeek'].min()} to {weekly_agg['YearWeek'].max()}")
    print(f"- Records with sales: {(weekly_agg['Weekly_Quantity_Sold'] > 0).sum()}")
    print(f"- Records with zero sales: {(weekly_agg['Weekly_Quantity_Sold'] == 0).sum()}")
    print(f"- Total quantity sold: {weekly_agg['Weekly_Quantity_Sold'].sum():,.0f}")
    print(f"- Total revenue: £{weekly_agg['Weekly_Total_Price'].sum():,.2f}")
    
    # Show distribution of per unit prices (excluding zeros)
    monthly_prices = monthly_agg[monthly_agg['Monthly_Per_Unit_Price'] > 0]['Monthly_Per_Unit_Price']
    weekly_prices = weekly_agg[weekly_agg['Weekly_Per_Unit_Price'] > 0]['Weekly_Per_Unit_Price']
    
    if len(monthly_prices) > 0:
        print(f"\nMonthly Per Unit Price Stats (excluding zeros):")
        print(f"- Average: £{monthly_prices.mean():.2f}")
        print(f"- Median: £{monthly_prices.median():.2f}")
        print(f"- Min: £{monthly_prices.min():.2f}")
        print(f"- Max: £{monthly_prices.max():.2f}")
    
    if len(weekly_prices) > 0:
        print(f"\nWeekly Per Unit Price Stats (excluding zeros):")
        print(f"- Average: £{weekly_prices.mean():.2f}")
        print(f"- Median: £{weekly_prices.median():.2f}")
        print(f"- Min: £{weekly_prices.min():.2f}")
        print(f"- Max: £{weekly_prices.max():.2f}")

# Implementing 

df = pd.read_csv('E:\sales_analysis\datasets\cleaned_data_with_filled_descriptions.csv')

monthly_data, weekly_data = create_aggregated_datasets(df)

analyze_aggregated_data(monthly_data, weekly_data)

monthly_data.to_csv('E:\sales_analysis\datasets\monthly_sales_aggregated.csv', index=False)
weekly_data.to_csv('E:\sales_analysis\datasets\weekly_sales_aggregated.csv', index=False)

<>:186: SyntaxWarning: invalid escape sequence '\s'
<>:192: SyntaxWarning: invalid escape sequence '\s'
<>:193: SyntaxWarning: invalid escape sequence '\s'
<>:186: SyntaxWarning: invalid escape sequence '\s'
<>:192: SyntaxWarning: invalid escape sequence '\s'
<>:193: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6008\1866671278.py:186: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv('E:\sales_analysis\datasets\cleaned_data_with_filled_descriptions.csv')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6008\1866671278.py:192: SyntaxWarning: invalid escape sequence '\s'
  monthly_data.to_csv('E:\sales_analysis\datasets\monthly_sales_aggregated.csv', index=False)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6008\1866671278.py:193: SyntaxWarning: invalid escape sequence '\s'
  weekly_data.to_csv('E:\sales_analysis\datasets\weekly_sales_aggregated.csv', index=False)


Processing 525096 transactions...
Date range: 2009-12-01 07:45:00 to 2010-12-09 20:01:00
Unique products: 4276
Complete period range: 2009-12-01 to 2010-12-31
Total months to cover: 13
Total weeks to cover: 57

Creating monthly aggregation...
Monthly aggregation created: 55588 records
Records with zero sales: 19363

Creating weekly aggregation...
Weekly aggregation created: 243732 records
Records with zero sales: 140860

=== AGGREGATION ANALYSIS ===

Monthly Data:
- Total product-month combinations: 55588
- Unique products: 4276
- Date range: 2009-12 to 2010-12
- Records with sales: 34756
- Records with zero sales: 19363
- Total quantity sold: 5,445,094
- Total revenue: £9,539,484.63

Weekly Data:
- Total product-week combinations: 243732
- Unique products: 4276
- Date range: 2009-11-30/2009-12-06 to 2010-12-27/2011-01-02
- Records with sales: 100123
- Records with zero sales: 140860
- Total quantity sold: 5,445,094
- Total revenue: £9,539,484.63

Monthly Per Unit Price Stats (excludin

In [7]:
# verifying number of datapoints

dfh = pd.read_csv('E:\sales_analysis\datasets\monthly_sales_aggregated.csv')

unique_products = dfh["StockCode"].unique()

print("TOTAL UNIQUE PRODUCTS :", len(unique_products))

print(55588/13)

TOTAL UNIQUE PRODUCTS : 4276
4276.0


<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6008\3476065413.py:3: SyntaxWarning: invalid escape sequence '\s'
  dfh = pd.read_csv('E:\sales_analysis\datasets\monthly_sales_aggregated.csv')


Objective 3 : Removing unnecessary datapoints from 'monthly_sales_aggregated.csv', 'weekly_sales_aggregated.csv', and 'product_sales_summary.csv'

List of Unnecessary DataPoint (Entire Row) (these values are in Invoice column):
* DOT  
* D  
* C2  
* BANK CHARGES  
* B  
* AMAZONFEE  
* ADJUST2  
* ADJUST  
* S  
* POST  
* M  
* m
* gift_0001_10
* gift_0001_20
* gift_0001_30
* gift_0001_40
* gift_0001_50
* gift_0001_60
* gift_0001_70
* gift_0001_80

Method :

-> Remove the entire row whose 'Invoice' value is from the list given above!

In [8]:
def remove_unnecessary_datapoints(file_paths, invoice_column='Invoice'):
    """
    Remove unnecessary datapoints from aggregated sales CSV files based on Invoice values.
    
    Parameters:
    file_paths (list): List of CSV file paths to clean
    stockcode_column (str): Name of the stock code column (default: 'StockCode')
    
    Returns:
    dict: Summary of cleaning results for each file
    """
    
    # List of unnecessary datapoint values to remove
    unnecessary_values = [
        'DOT', 'D', 'C2', 'BANK CHARGES', 'B', 'AMAZONFEE', 
        'ADJUST2', 'ADJUST', 'S', 'POST', 'M', 'm',
        'gift_0001_10', 'gift_0001_20', 'gift_0001_30', 'gift_0001_40',
        'gift_0001_50', 'gift_0001_60', 'gift_0001_70', 'gift_0001_80'
    ]
    
    print("=== REMOVING UNNECESSARY DATAPOINTS ===")
    print(f"Unnecessary values to remove: {unnecessary_values}")
    print(f"Files to process: {len(file_paths)}")
    
    results_summary = {}
    
    for file_path in file_paths:
        print(f"\n--- Processing: {file_path} ---")
        
        try:
            # Load the CSV file
            df = pd.read_csv(file_path)
            original_rows = len(df)
            
            print(f"Original rows: {original_rows}")
            
            # Check if Invoice column exists
            if invoice_column not in df.columns:
                print(f"WARNING: Column '{invoice_column}' not found in {file_path}")
                print(f"Available columns: {list(df.columns)}")
                results_summary[file_path] = {
                    'status': 'column_not_found',
                    'original_rows': original_rows,
                    'removed_rows': 0,
                    'final_rows': original_rows,
                    'available_columns': list(df.columns)
                }
                continue
            
            # Count occurrences of each unnecessary value before removal
            print("Unnecessary values found:")
            removal_counts = {}
            for value in unnecessary_values:
                count = (df[invoice_column] == value).sum()
                if count > 0:
                    removal_counts[value] = count
                    print(f"  {value}: {count} rows")
            
            if not removal_counts:
                print("  No unnecessary values found in this file")
            
            # Remove rows with unnecessary values
            mask = ~df[invoice_column].isin(unnecessary_values)
            df_cleaned = df[mask].copy()
            
            removed_rows = original_rows - len(df_cleaned)
            final_rows = len(df_cleaned)
            
            print(f"Rows removed: {removed_rows}")
            print(f"Final rows: {final_rows}")
            print(f"Percentage removed: {(removed_rows/original_rows)*100:.2f}%")
            
            # Save the cleaned file (backup original first)
            backup_path = file_path.replace('.csv', '_backup.csv')
            df.to_csv(backup_path, index=False)
            print(f"Original file backed up as: {backup_path}")
            
            # Save cleaned file
            df_cleaned.to_csv(file_path, index=False)
            print(f"Cleaned file saved: {file_path}")
            
            # Store results
            results_summary[file_path] = {
                'status': 'success',
                'original_rows': original_rows,
                'removed_rows': removed_rows,
                'final_rows': final_rows,
                'removal_details': removal_counts
            }
            
        except Exception as e:
            print(f"ERROR processing {file_path}: {e}")
            results_summary[file_path] = {
                'status': 'error',
                'error_message': str(e)
            }
    
    return results_summary

def clean_sales_files(monthly_file='datasets/monthly_sales_aggregated.csv', 
                     weekly_file='datasets/weekly_sales_aggregated.csv',
                     summary_file='datasets/product_sales_summary.csv',
                     stockcode_column='StockCode'):
    """
    Convenience function to clean the standard sales aggregation files.
    
    Parameters:
    monthly_file (str): Path to monthly sales file
    weekly_file (str): Path to weekly sales file  
    summary_file (str): Path to product sales summary file
    stockcode_column (str): Name of the stock code column
    
    Returns:
    dict: Cleaning results summary
    """
    
    file_paths = [monthly_file, weekly_file, summary_file]

    return remove_unnecessary_datapoints(file_paths, stockcode_column)

def print_cleaning_summary(results_summary):
    """
    Print a comprehensive summary of the cleaning results.
    
    Parameters:
    results_summary (dict): Results from remove_unnecessary_datapoints()
    """
    
    print("\n" + "="*60)
    print("CLEANING SUMMARY")
    print("="*60)
    
    total_original = 0
    total_removed = 0
    total_final = 0
    successful_files = 0
    
    for file_path, results in results_summary.items():
        print(f"\nFile: {file_path}")
        print(f"Status: {results['status']}")
        
        if results['status'] == 'success':
            successful_files += 1
            total_original += results['original_rows']
            total_removed += results['removed_rows']
            total_final += results['final_rows']
            
            print(f"  Original rows: {results['original_rows']:,}")
            print(f"  Removed rows: {results['removed_rows']:,}")
            print(f"  Final rows: {results['final_rows']:,}")
            print(f"  Removal rate: {(results['removed_rows']/results['original_rows'])*100:.2f}%")
            
            if results['removal_details']:
                print("  Removed values breakdown:")
                for value, count in results['removal_details'].items():
                    print(f"    {value}: {count:,} rows")
        
        elif results['status'] == 'file_not_found':
            print("  File not found - skipped")
        
        elif results['status'] == 'column_not_found':
            print(f"  Invoice column not found")
            print(f"  Available columns: {results.get('available_columns', [])}")
        
        elif results['status'] == 'error':
            print(f"  Error: {results['error_message']}")
    
    if successful_files > 0:
        print(f"\n" + "="*60)
        print("OVERALL SUMMARY")
        print("="*60)
        print(f"Files successfully processed: {successful_files}")
        print(f"Total original rows: {total_original:,}")
        print(f"Total removed rows: {total_removed:,}")
        print(f"Total final rows: {total_final:,}")
        print(f"Overall removal rate: {(total_removed/total_original)*100:.2f}%")

# Main execution
if __name__ == "__main__":
    
    
    print("Ready to clean sales aggregation files!")
    print("\nTo run the cleaning process, use one of these commands:")
    print("1. results = clean_sales_files()")
    print("2. print_cleaning_summary(results)")
    
    
    results = clean_sales_files()
    print_cleaning_summary(results)

Ready to clean sales aggregation files!

To run the cleaning process, use one of these commands:
1. results = clean_sales_files()
2. print_cleaning_summary(results)
=== REMOVING UNNECESSARY DATAPOINTS ===
Unnecessary values to remove: ['DOT', 'D', 'C2', 'BANK CHARGES', 'B', 'AMAZONFEE', 'ADJUST2', 'ADJUST', 'S', 'POST', 'M', 'm', 'gift_0001_10', 'gift_0001_20', 'gift_0001_30', 'gift_0001_40', 'gift_0001_50', 'gift_0001_60', 'gift_0001_70', 'gift_0001_80']
Files to process: 3

--- Processing: datasets/monthly_sales_aggregated.csv ---
ERROR processing datasets/monthly_sales_aggregated.csv: [Errno 2] No such file or directory: 'datasets/monthly_sales_aggregated.csv'

--- Processing: datasets/weekly_sales_aggregated.csv ---
ERROR processing datasets/weekly_sales_aggregated.csv: [Errno 2] No such file or directory: 'datasets/weekly_sales_aggregated.csv'

--- Processing: datasets/product_sales_summary.csv ---
ERROR processing datasets/product_sales_summary.csv: [Errno 2] No such file or dire

In [9]:
# Number of unique customers

unique_customers = df["Customer ID"].unique()

print("TOTAL UNIQUE CUSTOMERS :", len(unique_customers))

TOTAL UNIQUE CUSTOMERS : 4384
